In [ ]:
!pip install -Uqq fastbook
import fastbook
try:
  fastbook.setup_book()
except:
  print("No password entered")

     |████████████████████████████████| 727kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 21.7MB/s 
     |████████████████████████████████| 194kB 46.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 40kB 5.1MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 2.6MB 39.5MB/s 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:



No password entered


In [ ]:
from fastbook import *
from IPython.display import display, HTML

## NLP introduction

nlp is about guessing the next word. for that the model needs to create its own labels. It uses self supervised learning.

The language model used to classify IMDb was pretrained on Wikipedia. 

why learn in detail ?

One reason, of course, is that it is helpful to understand the foundations of the models that you are using. But there is another very practical reason, which is that you get even better results if you fine-tune the (sequence-based) language model prior to fine-tuning the classification model

We will be finetuning the pretrained language model which was trained on wikipedia articles.

This is called ULMFit approach.

### text preprocessing

we already know how categorical variables can be used as independent variables for neural network

make a list of all possible levels of the variable (vocab)

Replace each level with its index in the vocab

create an embedding matrix for this contatining a row for each level i.e. for each item in the vocab

Use this embedding matrix as the first layer of a neural network.

we do the same thing with text. 

whats new is idea of sequence. first we concatenate all of the document in our dataset into onw big long string and split it into words giving us very long words or tokens. 

Our independent variable will be the entire string exect for the second last and last will be labe;

Our vocab would be mix of common words from wikipedia and new words specific to our corpus would be movie actors

for building embedding matrix: for words in vocabualary of pre trained modelwe will take corresponding row in the embedding matrix of the pretrained model but for new words we won't have anythong, we willjust initialize the corresponding row with a random vector

jargon

tokenisation

Numericalisations - making list of unique words -vocab and convert each word to index to look up in vocab

Language model data loader creation -  LMDDataLoader class for seperating the last token as label

Language model creation - creating a model that handles the input list that are arbitaryily small or big.

### Tokenisation

converting to words
 word based - split a sentence on spaces and apply language specific rules to try to seperate parts of meaning even when there are no spaces

 subword - word is split like occ aa sion

 Character based - individual character

 with fastai word tokenisation is done through exernal API

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

need to gradb the text file, like get_imagefiles we have get_text_file.

we can also pass folder to restrict the search

In [ ]:
!dir {path}

imdb.vocab  README  test  tmp_clas  tmp_lm  train  unsup


In [ ]:
files = get_text_files(path, folders=['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read()
txt[:75]

'There are a number of things that are not correct, although this is not too'

fastai by default used spacyTokenizer

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(toks)

['There', 'are', 'a', 'number', 'of', 'things', 'that', 'are', 'not', 'correct', ',', 'although', 'this', 'is', 'not', 'too', 'important', 'since', 'what', 'happened', 'to', 'whom', 'and', 'when', 'is', 'still', 'in', 'dispute', '.', 'The', 'most', 'blatant', 'liberty', 'with', 'the', 'facts', 'I', 'think', 'is', 'when', 'they', 'start', 'to', 'play', 'at', 'Bruno', 'Koschmidder', "'s", 'Kaiserkeller', ',', 'when', 'in', 'fact', 'they', 'played', 'at', 'the', 'Indra', 'and', 'moved', 'to', 'the', 'Kaiserkeller', 'later.<br', '/><br', '/>I', 'agree', 'with', 'Semprinni20', 'that', 'the', 'film', 'was', 'biased', 'in', 'favour', 'of', 'Pete', 'Best', "'s", 'version', ',', 'but', 'if', 'he', 'is', 'the', 'story', 'consultant', 'then', 'I', 'guess', 'he', 'calls', 'the', 'shots', '.', 'I', 'also', 'agree', 'with', 'Semprinni', 'that', 'the', 'recordings', 'Pete', 'Best', 'plays', 'on', 'say', 'the', 'last', 'word', 'on', 'the', 'subject', 'of', 'why', 'he', 'was', 'fired.<br', '/><br', '/>

In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00']))

(#8) ['The','U.S.','dollar','$','1','is','$','1.00']

In [ ]:
# some additional functionality is added to Tokenizer class

tkn = Tokenizer(spacy)

print(tkn(txt))

# xxbos indicates beginning of the stream

['xxbos', 'xxmaj', 'there', 'are', 'a', 'number', 'of', 'things', 'that', 'are', 'not', 'correct', ',', 'although', 'this', 'is', 'not', 'too', 'important', 'since', 'what', 'happened', 'to', 'whom', 'and', 'when', 'is', 'still', 'in', 'dispute', '.', 'xxmaj', 'the', 'most', 'blatant', 'liberty', 'with', 'the', 'facts', 'i', 'think', 'is', 'when', 'they', 'start', 'to', 'play', 'at', 'xxmaj', 'bruno', 'xxmaj', 'koschmidder', "'s", 'xxmaj', 'kaiserkeller', ',', 'when', 'in', 'fact', 'they', 'played', 'at', 'the', 'xxmaj', 'indra', 'and', 'moved', 'to', 'the', 'xxmaj', 'kaiserkeller', 'later', '.', '\n\n', 'i', 'agree', 'with', 'xxmaj', 'semprinni20', 'that', 'the', 'film', 'was', 'biased', 'in', 'favour', 'of', 'xxmaj', 'pete', 'xxmaj', 'best', "'s", 'version', ',', 'but', 'if', 'he', 'is', 'the', 'story', 'consultant', 'then', 'i', 'guess', 'he', 'calls', 'the', 'shots', '.', 'i', 'also', 'agree', 'with', 'xxmaj', 'semprinni', 'that', 'the', 'recordings', 'xxmaj', 'pete', 'xxmaj', 'bes

In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [ ]:
#??replace_rep

In [ ]:
coll_repr(tkn('&copy; Fast.ai ww.fast.ai/INDEX'), 31)

"(#8) ['xxbos','©','xxmaj','fast.ai','ww.fast.ai','/','xxup','index']"

### subword tokenisation

Used especially in language like chinese as they have no spaces

then we 
1. analyse the corpus for most commonly occuring grouping
2. tokenize using the vocab of subword units

In [ ]:
txts = L(o.open().read() for o in files[:2000])
# L behaves like a list of items with extra functionalities

In [ ]:
txts

(#2000) ['There are a number of things that are not correct, although this is not too important since what happened to whom and when is still in dispute. The most blatant liberty with the facts I think is when they start to play at Bruno Koschmidder\'s Kaiserkeller, when in fact they played at the Indra and moved to the Kaiserkeller later.<br /><br />I agree with Semprinni20 that the film was biased in favour of Pete Best\'s version, but if he is the story consultant then I guess he calls the shots. I also agree with Semprinni that the recordings Pete Best plays on say the last word on the subject of why he was fired.<br /><br />Although the film is not such a lavish production as the later film "Backbeat", I prefer this film because it is more accurate, and because it has a better script with deeper characterisation.<br /><br />There is plenty in the film that is quite substantial - such as Brian Epstein trying to hide the fact that he has been "queer-bashed," only to find out that th

we instantiate the tokeniser passing in the size of vocab and then find the common sequences of characters to create the vocab. this is done with setup. it is called automatically but we are doing ti here

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txts]))[:40])



In [ ]:
sp = SubwordTokenizer(vocab_sz=1000)
sp.setup(txts)
print(sp([txts]))
for i in str(sp([txts])):
  print(str(i))

<generator object SentencePieceTokenizer.__call__ at 0x7f4171b0ca98>
<
g
e
n
e
r
a
t
o
r
 
o
b
j
e
c
t
 
S
e
n
t
e
n
c
e
P
i
e
c
e
T
o
k
e
n
i
z
e
r
.
_
_
c
a
l
l
_
_
 
a
t
 
0
x
7
f
4
1
7
1
b
0
c
a
9
8
>


In [ ]:
#subword(1000)

If er use samller vocab then each token would be smaller bbut it will take more tokens to represent a sentence.

I its large then most common phrases themselves would be in vocab

subwords are important because then we can also train on genomic sequence and MIDI music data

### Numericalisation

IN this we map the token to integers. It is similar to steps needded to create a categorical model.

in this we make a list of all possible levels of that categorical variable

Replace each level with index in vocab

In [ ]:
toks = tkn(txt)

print(coll_repr(tkn(txt), 5))

(#314) ['xxbos','xxmaj','there','are','a'...]


in orderto use numericalise we need to call setup

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[199]

(#131) ['xxbos','"','after','dark',',','my','sweet','"','is','a'...]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#2184) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','that'...]"

thedefaults to Numericalize are min_freq=2 and max _vocab=60000 this results in fastai replacing all words other than the most common 60 000 with a special unknown word token xxunk.

this is to avoid overly large embedding matrix

after creating num we can use it like a token

In [ ]:
nums = num(toks)[:20]
nums

tensor([  2,   8,  67,  41,  13, 440,  14, 207,  19,  41,  42,   0,  10, 281,  20,  16,  42, 130, 515, 247])

they can be mapped back intooriginal text

In [ ]:
num.vocab[20]

'this'

### Putting text into batches for a language Model

two considerations : irregular length and importance of begining when the previous batch left off

In [ ]:
stream = "In this chapter, we will go over what has been done previously and try to be the best thing that ever got into the the history of these leaves. To be frank I never expected me to be the kind of guy who would follow through. But what do you kno here we are doing what we do worst. You can't be at your best everytime, however the problem is I am at my worst more time htan I would like. And to make matters worse I rarely learn."
tokens = tkn(stream)
bs, seq_len = 6, 15
d_tokens = np.array( [ tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i


we cant simple give the above words to model because of GPU restrictions.

Sowe neeedto devide this array more finely into subarrays of fixed sequence length

In [ ]:
# if we want a sequence len of five for 6 batches of length 15

bs, seq_len = 6, 5
for j in range(0,15):
  d_tokens = np.array([tokens[i*15:i*15+j * seq_len] for i in range(bs)])
  df = pd.DataFrame(d_tokens)
  display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,in,this,chapter
and,try,to,be,the
these,leaves,.,xxmaj,to
of,guy,who,would,follow
are,doing,what,we,do
everytime,",",however,the,problem


xxbos,xxmaj,in,this,chapter,",",we,will,go,over
and,try,to,be,the,best,thing,that,ever,got
these,leaves,.,xxmaj,to,be,frank,i,never,expected
of,guy,who,would,follow,through,.,xxmaj,but,what
are,doing,what,we,do,worst,.,xxmaj,you,ca
everytime,",",however,the,problem,is,i,am,at,my


xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i


xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously,and,try,to,be,the
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of,these,leaves,.,xxmaj,to
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind,of,guy,who,would,follow
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we,are,doing,what,we,do
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best,everytime,",",however,the,problem
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i,would,like,.,xxmaj,and


xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously,and,try,to,be,the,best,thing,that,ever,got
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of,these,leaves,.,xxmaj,to,be,frank,i,never,expected
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind,of,guy,who,would,follow,through,.,xxmaj,but,what
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we,are,doing,what,we,do,worst,.,xxmaj,you,ca
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best,everytime,",",however,the,problem,is,i,am,at,my
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i,would,like,.,xxmaj,and,to,make,matters,worse,i


...

"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


we need to transform texts inot a stream by concatenating them together. As with images it is best to randomize order of documents (not the text!)

Then we cut the stream to batch which is our batch size. for instance if stream is 50000 then we set a batch size of 10 this will give us 10 mini streams of 5000 tokens

this is the behindthe scenes for fastai librart when we create LMDataLoader. we do it when we apply Numericalize


In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
# grabbing the first batch
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [ ]:
x[0][:20]

tensor([  2,   8,  67,  41,  13, 440,  14, 207,  19,  41,  42,   0,  10, 281,  20,  16,  42, 130, 515, 247])

In [ ]:
' '.join([num.vocab[o] for o in x[0][:20]])

'xxbos xxmaj there are a number of things that are not xxunk , although this is not too important since'

### Training a Text classifier

